<a href="https://colab.research.google.com/github/erezimm/ML-ML/blob/master/GNN_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Main notebook, to be run on Google Colab

## Import dependencies and download dataset

In [ ]:
!pip install dgl

import glob
from datetime import datetime
import os
import shutil

import matplotlib.pyplot as plt
import astropy.io.ascii
import dgl
import networkx as nx
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm

if not torch.cuda.is_available():
    raise Exception('Cuda unavailable, turn it on under Runtime>Change runtime type>GPU')
device = torch.device('cpu')

training_path = '/content/training_set'
validation_path = '/content/validation_set'
for path in (training_path, validation_path):
    if not os.path.isdir(path):
        os.makedirs(path)

!wget -O data.zip https://github.com/erezimm/ML-ML/raw/master/data.zip
!unzip -o -q data.zip -d /content/
data_path = '/content/data'
microlist = glob.glob(data_path+'/microlensedconst_*')
varlist = glob.glob(data_path+'/cleanvar_*')
for typelist in (microlist, varlist):
    for i, f in enumerate(typelist):
        dest = training_path if i<len(microlist)/2 else validation_path
        shutil.move(f, os.path.join(dest, os.path.basename(f)))

## Create Datasets
Class CustomDataset loads all files and converts them to a list of graphs.
Each node in a graph is a data point, i.e. node features are days since start, mag, magerr

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, path):
        filelist = glob.glob(path+'/*')
        
        self.graphs = []
        self.isMicrolensed = []
        for fname in tqdm(filelist):
            filebasename = os.path.basename(fname)
            if filebasename.startswith('cleanvar'):
                microlensed = torch.tensor([0])
            elif filebasename.startswith('microlensedconst'):
                microlensed = torch.tensor([1])
            else:
                raise Exception('Filename ' + fname + ' not clean_* or microlensed_*')
            
            with open(fname) as f:
                data = astropy.io.ascii.read(f.read())
                times, mags, magerrs = data['col1'], [], data['col3']
                for m, err in zip(data['col2'], magerrs):
                    mags.append(round(m, len(str(err).split('.')[1])))
                zipped = list(zip(times, mags, magerrs))
                zipped.sort(key=lambda tup: tup[0])
                times, mags, magerrs = zip(*zipped)

                n = len(times)
                g = dgl.DGLGraph()
                g.add_nodes(n)
                g.ndata['time'] = torch.tensor(times).float()  # days
                g.ndata['mag'] = torch.tensor(mags).float()
                g.ndata['magerr'] = torch.tensor(magerrs).float()
                g.add_edges([i for i in range(n)], [(i+1)%n for i in range(n)])
                g.add_edges([i for i in range(n)], [(i-1)%n for i in range(n)])
                
                self.graphs.append(g)
                self.isMicrolensed.append(microlensed)
        
    def __len__(self):
        return len(self.graphs)

    def __getitem__(self, idx):
        return self.graphs[idx], self.isMicrolensed[idx]


train_ds = CustomDataset(training_path)
validation_ds = CustomDataset(validation_path)

nx.draw(dgl.to_networkx(train_ds[20][0]))

In [ ]:
def collate(samples):
    # The input `samples` is a list, a batch of whatever comes out of your dataset object
    graphs = [x[0] for x in samples]
    labels = [x[1] for x in samples]
    
    batched_graph = dgl.batch(graphs, node_attrs=['time', 'mag', 'magerr'])
    targets = torch.cat(labels)
    
    return batched_graph, targets.unsqueeze(1).float()

## The Newtwork

In [ ]:
n_ft = 3  # number of node features (time, mag, magerr)

class EdgeNetwork(nn.Module):
    def __init__(self, node_hidrep, edge_hidrep):
        super(EdgeNetwork, self).__init__()
        n_in = 2*node_hidrep
        
        # network:
        sizes = [n_in] + [] + [edge_hidrep]

        layers = []
        for i in range(len(sizes)-1):
            layers.extend([nn.Linear(sizes[i], sizes[i+1]), nn.ReLU()])
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        input = torch.cat((x.dst['node_hidrep'], x.src['node_hidrep']), dim=1)
        output = self.network(input)
        return {'edge_hidrep': output }

    
class NodeNetwork(nn.Module):
    def __init__(self, node_hidrep, edge_hidrep):
        super(NodeNetwork, self).__init__()
        n_in = edge_hidrep + node_hidrep

        # network:
        sizes = [n_in] + [] + [node_hidrep]

        layers = []
        for i in range(len(sizes)-1):
            layers.extend([nn.Linear(sizes[i], sizes[i+1]), nn.ReLU()])
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        mb = torch.mean( x.mailbox['edge_hidrep'] , dim=1 )
        input = torch.cat((mb, x.data['node_hidrep']), dim=1)
        out = self.network(input)
        return {'node_hidrep': out }


class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()

        # networks:
        node_hidrep, edge_hidrep = 10, 10
        sizes_nodeinit = [n_ft] + [] + [node_hidrep]
        self.update_loop_length = 20

        layers = []
        for i in range(len(sizes_nodeinit)-1):
            layers.extend([nn.Linear(sizes_nodeinit[i], sizes_nodeinit[i+1]), nn.ReLU()])
        self.node_init = nn.Sequential(*layers)
        self.edge_network = EdgeNetwork(node_hidrep, edge_hidrep)
        self.node_network = NodeNetwork(node_hidrep, edge_hidrep)
        
    def forward(self, g):
        features = torch.tensor(list(zip(g.ndata['time'], g.ndata['mag'], g.ndata['magerr'])))
        g.ndata['node_hidrep'] = self.node_init(features)

        for i in range(self.update_loop_length):
            g.update_all(self.edge_network, self.node_network)
        
        # eventually take max of all entries in all nodes (?), maybe you can apply functions with dgl.function ?
        output = dgl.max_nodes(g, 'node_hidrep')
        output = torch.max(output, 1, keepdim=True)[0]
        return output

In [ ]:
net = Classifier()
net.to(device)

## Train

In [ ]:
def evaluate(net, validation_ds):
    test_data_loader = DataLoader(validation_ds, batch_size=30, shuffle=True, collate_fn=collate)

    net.cpu()
    net.eval()

    true_positive = 0
    false_positive = 0
    true_negative = 0
    false_negative = 0
    total_positives = 0
    total_negatives = 0

    for i, (x,y) in enumerate(test_data_loader):
            y = y.data.numpy()
            prediction = net(x).cpu().data.numpy()
            
            prediction[prediction >= 0.5] = 1
            prediction[prediction <0.5] = 0
            
            total_positives+=len(np.where( y==1 )[0])
            total_negatives+=len(np.where( y==0 )[0])
            
            true_positive+= len(np.where( (prediction==y) & (y==1) )[0])
            # true_negative+= len(np.where( (prediction==y) & (y==0) )[0])
            false_positive+= len(np.where( (prediction!=y) & (y==0) )[0])
            # false_negative+= len(np.where( (prediction!=y) & (y==1) )[0])
                    
    print('Probability of detection:', true_positive/total_positives)  # probability of recognizing a microlensing when you see it
    print('Probability of false alarm:', false_positive/total_negatives)  # probability for misclassification of a variable star as microlensing

In [ ]:
# options:
loss_func = nn.BCEWithLogitsLoss()
# optimizer = optim.SGD(net.parameters(), lr=0.01)
optimizer = optim.Adam(net.parameters(), lr=0.001)
n_epochs = 50

losses = []

data_loader = DataLoader(train_ds, batch_size=30, shuffle=True, collate_fn=collate)
for epoch in tqdm(range(n_epochs)):
    net.train()
    this_losses = []
    for x,y in data_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        output = net(x)
        # print(output.shape, y.shape)
        loss = loss_func(output,y)
        this_losses.append(float(loss))
        loss.backward()
        optimizer.step()

    losses.append(np.mean(this_losses))
    print('Epoch:', epoch, 'Loss:', round(losses[-1], 4))  # for some real-time indication (couldn't manage to get this to dynamically plot)

In [ ]:
# plot the printed losses:
losses = np.array(losses)
plt.plot(losses[1:],label='training loss')
plt.xlabel('Epoch')
plt.legend()
plt.grid()
plt.show()

evaluate(net, validation_ds)  # show P_D, P_FA

## Save
Don't forget to download

In [ ]:
timestamp = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
torch.save(net.state_dict(), 'model_' + timestamp + '.pt')